In [ ]:
#@title <h1>Requirements</h1>

!pip install os-sys
import os

# Path to the output file
output_file_path = 'path/to/output/file.txt'

# Delete existing output file before processing, if any
if os.path.exists(output_file_path):
    os.remove(output_file_path)

# Rest of your code


In [ ]:
#@title <h1>Setup Wav2Lip</h1>
#@markdown * Install dependency
#@markdown * Download pretrained model
from IPython.display import HTML, clear_output
!rm -rf /content/sample_data
!mkdir /content/sample_data

!git clone https://github.com/justinjohn0306/Wav2Lip

%cd /content/Wav2Lip

#download the pretrained model
!wget 'https://github.com/justinjohn0306/Wav2Lip/releases/download/models/wav2lip.pth' -O 'checkpoints/wav2lip.pth'
!wget 'https://github.com/justinjohn0306/Wav2Lip/releases/download/models/wav2lip_gan.pth' -O 'checkpoints/wav2lip_gan.pth'
!wget 'https://github.com/justinjohn0306/Wav2Lip/releases/download/models/resnet50.pth' -O 'checkpoints/resnet50.pth'
!wget 'https://github.com/justinjohn0306/Wav2Lip/releases/download/models/mobilenet.pth' -O 'checkpoints/mobilenet.pth'
a = !pip install https://raw.githubusercontent.com/AwaleSajil/ghc/master/ghc-1.0-py3-none-any.whl
!pip install git+https://github.com/elliottzheng/batch-face.git@master

!pip install ffmpeg-python mediapipe==0.8.11

#this code for recording audio
"""
To write this piece of code I took inspiration/code from a lot of places.
It was late night, so I'm not sure how much I created or just copied o.O
Here are some of the possible references:
https://blog.addpipe.com/recording-audio-in-the-browser-using-pure-html5-and-minimal-javascript/
https://stackoverflow.com/a/18650249
https://hacks.mozilla.org/2014/06/easy-audio-capture-with-the-mediarecorder-api/
https://air.ghost.io/recording-to-an-audio-file-using-html5-and-js/
https://stackoverflow.com/a/49019356
"""
from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import read as wav_read
import io
!pip install ffmpeg-python
import ffmpeg

AUDIO_HTML = """
<script>
var my_div = document.createElement("DIV");
var my_p = document.createElement("P");
var my_btn = document.createElement("BUTTON");
var t = document.createTextNode("Press to start recording");

my_btn.appendChild(t);
//my_p.appendChild(my_btn);
my_div.appendChild(my_btn);
document.body.appendChild(my_div);

var base64data = 0;
var reader;
var recorder, gumStream;
var recordButton = my_btn;

var handleSuccess = function(stream) {
  gumStream = stream;
  var options = {
    //bitsPerSecond: 8000, //chrome seems to ignore, always 48k
    mimeType : 'audio/webm;codecs=opus'
    //mimeType : 'audio/webm;codecs=pcm'
  };
  //recorder = new MediaRecorder(stream, options);
  recorder = new MediaRecorder(stream);
  recorder.ondataavailable = function(e) {
    var url = URL.createObjectURL(e.data);
    var preview = document.createElement('audio');
    preview.controls = true;
    preview.src = url;
    document.body.appendChild(preview);

    reader = new FileReader();
    reader.readAsDataURL(e.data);
    reader.onloadend = function() {
      base64data = reader.result;
      //console.log("Inside FileReader:" + base64data);
    }
  };
  recorder.start();
  };

recordButton.innerText = "Recording... press to stop";

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);


function toggleRecording() {
  if (recorder && recorder.state == "recording") {
      recorder.stop();
      gumStream.getAudioTracks()[0].stop();
      recordButton.innerText = "Saving the recording... pls wait!"
  }
}

// https://stackoverflow.com/a/951057
function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}

var data = new Promise(resolve=>{
//recordButton.addEventListener("click", toggleRecording);
recordButton.onclick = ()=>{
toggleRecording()

sleep(2000).then(() => {
  // wait 2000ms for the data to be available...
  // ideally this should use something like await...
  //console.log("Inside data:" + base64data)
  resolve(base64data.toString())

});

}
});

</script>
"""

%cd /
from ghc.l_ghc_cf import l_ghc_cf
%cd content

def get_audio():
  display(HTML(AUDIO_HTML))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])

  process = (ffmpeg
    .input('pipe:0')
    .output('pipe:1', format='wav')
    .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
  )
  output, err = process.communicate(input=binary)

  riff_chunk_size = len(output) - 8
  # Break up the chunk size into four bytes, held in b.
  q = riff_chunk_size
  b = []
  for i in range(4):
      q, r = divmod(q, 256)
      b.append(r)

  # Replace bytes 4:8 in proc.stdout with the actual size of the RIFF chunk.
  riff = output[:4] + bytes(b) + output[8:]

  sr, audio = wav_read(io.BytesIO(riff))

  return audio, sr


from IPython.display import HTML
from base64 import b64encode
def showVideo(path):
  mp4 = open(str(path),'rb').read()
  data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
  return HTML("""
  <video width=700 controls>
        <source src="%s" type="video/mp4">
  </video>
  """ % data_url)

from IPython.display import clear_output

clear_output()
print("All set and ready!")

All set and ready!


In [ ]:
#@title <h1>Video Upload</h1>

import os
from google.colab import drive
from google.colab import files
from IPython.display import HTML, clear_output
from base64 import b64encode
import moviepy.editor as mp

def show_video(file_path):
    """Function to display video in Colab"""
    mp4 = open(file_path, 'rb').read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
    display(HTML("""
    <video controls width=600>
        <source src="%s" type="video/mp4">
    </video>
    """ % data_url))

def get_video_resolution(video_path):
    """Function to get the resolution of a video"""
    import cv2
    video = cv2.VideoCapture(video_path)
    width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
    video.release()
    return (width, height)

# Mount Google Drive if it's not already mounted
if not os.path.isdir("/content/drive/MyDrive"):
    drive.mount('/content/drive', force_remount=True)

#@markdown ### Select an uploading method
upload_method = "Upload"  #@param ["Upload", "Custom Path"]

input_video_path = '/content/sample_data/input_vid.mp4'

# Remove previous input video if exists
if os.path.isfile(input_video_path):
    os.remove(input_video_path)

if upload_method == "Upload":
    uploaded = files.upload()
    for filename in uploaded.keys():
        os.rename(filename, input_video_path)
    path_to_your_video = input_video_path

elif upload_method == 'Custom Path':
    #@markdown Add the full path to your video on your Gdrive
    path_to_your_video = '/content/drive/MyDrive/test.mp4'  #@param {type:"string"}
    if not os.path.isfile(path_to_your_video):
        print("ERROR: File not found!")
        raise SystemExit(0)

# Check video duration
video_duration = mp.VideoFileClip(path_to_your_video).duration
if video_duration > 60:
    print("WARNING: Video duration exceeds 60 seconds. Please upload a shorter video.")
    raise SystemExit(0)

# Check video resolution and resize if necessary
video_resolution = get_video_resolution(path_to_your_video)
print(f"Video resolution: {video_resolution}")
if video_resolution[0] >= 1920 or video_resolution[1] >= 1080:
    print("Resizing video to 1080p...")
    os.system(f"ffmpeg -i {path_to_your_video} -vf scale=1280:1080 {input_video_path}")
    path_to_your_video = input_video_path
    print("Video resized to 1080p")
else:
    print("No resizing needed")
    if upload_method == 'Custom Path' and path_to_your_video != input_video_path:
        os.rename(path_to_your_video, input_video_path)

# Clear output if uploaded
if upload_method == "Upload":
    clear_output()
    print("Input Video")
    show_video(input_video_path)
else:
    if os.path.isfile(input_video_path):
        print("Input Video")
        show_video(input_video_path)


  if event.key is 'enter':



Mounted at /content/drive


In [ ]:
#@title <h1>Audio Upload</h1>
import os
from IPython.display import Audio, clear_output
from IPython.core.display import display

upload_method = 'Upload'  #@param ['Record', 'Upload', 'Custom Path']

input_audio_path = '/content/sample_data/input_audio.wav'

# Remove previous input audio if exists
if os.path.isfile(input_audio_path):
    os.remove(input_audio_path)

def display_audio(file_path):
    """Function to display audio in Colab"""
    display(Audio(file_path))

if upload_method == 'Record':
    # Simulated recording function (replace with actual recording if needed)
    def get_audio():
        import numpy as np
        return np.random.randn(22050 * 5), 22050  # 5 seconds of random audio at 22050 Hz

    audio, sr = get_audio()

    from scipy.io.wavfile import write
    write(input_audio_path, sr, audio)

elif upload_method == 'Upload':
    from google.colab import files
    uploaded = files.upload()
    uploaded_file = next(iter(uploaded))  # Get the first uploaded file

    import librosa
    audio, sr = librosa.load(uploaded_file, sr=None)

    import soundfile as sf
    sf.write(input_audio_path, audio, sr, format='wav')

    clear_output()
    display_audio(input_audio_path)

else:  # Custom Path
    from google.colab import drive
    drive.mount('/content/drive')

    #@markdown Add the full path to your audio on your Gdrive
    path_to_your_audio = '/content/drive/MyDrive/test.wav'  #@param {type:"string"}

    if not os.path.isfile(path_to_your_audio):
        print("ERROR: File not found!")
        raise SystemExit(0)

    import librosa
    audio, sr = librosa.load(path_to_your_audio, sr=None)

    import soundfile as sf
    sf.write(input_audio_path, audio, sr, format='wav')

    clear_output()
    display_audio(input_audio_path)


In [ ]:
#@title <h1>Crunching and preview output</h1>
#@markdown <b>Note: Only change these, if you have to</b>

import os
from IPython.display import clear_output

# Set up paths and variables for the output file
output_file_path = '/content/Wav2Lip/results/result_voice.mp4'

# Delete existing output file before processing, if any
if os.path.exists(output_file_path):
    os.remove(output_file_path)

pad_top = 0  #@param {type:"integer"}
pad_bottom = 10  #@param {type:"integer"}
pad_left = 0  #@param {type:"integer"}
pad_right = 0  #@param {type:"integer"}
rescale_factor = 1  #@param {type:"integer"}
nosmooth = True  #@param {type:"boolean"}

#@markdown ___
#@markdown Model selection:
use_hd_model = True  #@param {type:"boolean"}
checkpoint_path = 'checkpoints/wav2lip.pth' if not use_hd_model else 'checkpoints/wav2lip_gan.pth'

# Navigate to Wav2Lip directory
os.chdir('/content/Wav2Lip')

# Build the inference command
inference_command = (
    f"python inference.py --checkpoint_path {checkpoint_path} "
    f"--face ../sample_data/input_vid.mp4 --audio ../sample_data/input_audio.wav "
    f"--pads {pad_top} {pad_bottom} {pad_left} {pad_right} --resize_factor {rescale_factor}"
)

if nosmooth:
    inference_command += " --nosmooth"

# Run the inference
os.system(inference_command)

# Preview output video
if os.path.exists(output_file_path):
    clear_output()
    print("Final Video Preview")
    print("Download this video from", output_file_path)
    show_video(output_file_path)
else:
    print("Processing failed. Output video not found.")


In [ ]:
import cv2
import numpy as np
from moviepy.editor import VideoFileClip, AudioFileClip
from base64 import b64encode
from IPython.display import HTML, display, clear_output
import ipywidgets as widgets

def showVideo(file_path):
    """Function to display video in Colab"""
    mp4 = open(file_path, 'rb').read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
    display(HTML("""
    <video controls width=600>
        <source src="%s" type="video/mp4">
    </video>
    """ % data_url))

def enhance_video(input_path, output_path):
    # Open the input video
    cap = cv2.VideoCapture(input_path)
    if not cap.isOpened():
        print("Error: Could not open video.")
        return

    # Get video properties
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    temp_output_path = 'temp_video.avi'
    out = cv2.VideoWriter(temp_output_path, fourcc, fps, (width, height))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Apply Gaussian Blur to reduce noise
        frame = cv2.GaussianBlur(frame, (5, 5), 0)

        # Sharpen the frame
        kernel = np.array([[0, -1, 0],
                           [-1, 5, -1],
                           [0, -1, 0]])
        frame = cv2.filter2D(frame, -1, kernel)

        # Convert to HSV and equalize histogram
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        hsv[:, :, 2] = cv2.equalizeHist(hsv[:, :, 2])
        frame = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

        # Adjust brightness and contrast
        alpha = 0.7  # Simple contrast control
        beta = 20    # Simple brightness control
        frame = cv2.convertScaleAbs(frame, alpha=alpha, beta=beta)

        out.write(frame)

    # Release the video objects
    cap.release()
    out.release()
    cv2.destroyAllWindows()

    # Combine the processed video with the original audio
    video = VideoFileClip(temp_output_path)
    audio = AudioFileClip(input_path)
    final_video = video.set_audio(audio)
    final_video.write_videofile(output_path, codec='libx264', audio_codec='aac')

# File upload widget
upload_widget = widgets.FileUpload(accept='.mp4', multiple=False)

def on_file_upload(change):
    # Get the uploaded file content
    uploaded_file = next(iter(change['new'].values()))
    file_path = 'uploaded_video.mp4'

    # Save the uploaded file
    with open(file_path, 'wb') as f:
        f.write(uploaded_file['content'])

    # Enhance video
    output_video_path = 'enhanced_video_f.mp4'
    enhance_video(file_path, output_video_path)
    showVideo(output_video_path)

upload_widget.observe(on_file_upload, names='value')
display(upload_widget)


In [ ]:
#@title <h1>Accuracy Finder</h1>
import cv2
from skimage.metrics import structural_similarity as ssim
import numpy as np
from google.colab import files

def calculate_ssim_psnr(predicted_video, ground_truth_video):
    predicted_frames = extract_frames(predicted_video)
    ground_truth_frames = extract_frames(ground_truth_video)

    ssim_values = []
    psnr_values = []

    # Resize frames to a common size
    common_size = (min(predicted_frames[0].shape[1], ground_truth_frames[0].shape[1]),
                   min(predicted_frames[0].shape[0], ground_truth_frames[0].shape[0]))

    for pred_frame, gt_frame in zip(predicted_frames, ground_truth_frames):
        pred_frame = cv2.resize(pred_frame, common_size)
        gt_frame = cv2.resize(gt_frame, common_size)

        pred_gray = cv2.cvtColor(pred_frame, cv2.COLOR_BGR2GRAY)
        gt_gray = cv2.cvtColor(gt_frame, cv2.COLOR_BGR2GRAY)

        ssim_value = ssim(pred_gray, gt_gray)
        ssim_values.append(ssim_value)

        mse = np.mean((pred_gray - gt_gray) ** 2)
        if mse == 0:
            psnr_value = float('inf')
        else:
            psnr_value = 20 * np.log10(255.0 / np.sqrt(mse))
        psnr_values.append(psnr_value)

    avg_ssim = np.mean(ssim_values)
    avg_psnr = np.mean(psnr_values)

    return avg_ssim, avg_psnr

def extract_frames(video_path):
    cap = cv2.VideoCapture(video_path)
    frames = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(frame)

    cap.release()
    return frames

# Upload predicted video
print("Please upload the predicted video:")
uploaded_predicted = files.upload()
predicted_video_path = list(uploaded_predicted.keys())[0]

# Upload ground truth video
print("Please upload the ground truth video:")
uploaded_ground_truth = files.upload()
ground_truth_video_path = list(uploaded_ground_truth.keys())[0]

# Calculate SSIM and PSNR
avg_ssim, avg_psnr = calculate_ssim_psnr(predicted_video_path, ground_truth_video_path)
print(f"Average SSIM: {avg_ssim}")
print(f"Average PSNR: {avg_psnr}")


In [ ]:
#@title <h1>Accuracy Finder and Graphical Representation</h1>
import cv2
from skimage.metrics import structural_similarity as ssim
import numpy as np
import matplotlib.pyplot as plt
from google.colab import files

def calculate_ssim_psnr_per_frame(predicted_video, ground_truth_video):
    predicted_frames = extract_frames(predicted_video)
    ground_truth_frames = extract_frames(ground_truth_video)

    ssim_values = []
    psnr_values = []

    # Determine the smallest common size
    common_size = (min(predicted_frames[0].shape[1], ground_truth_frames[0].shape[1]),
                   min(predicted_frames[0].shape[0], ground_truth_frames[0].shape[0]))

    for pred_frame, gt_frame in zip(predicted_frames, ground_truth_frames):
        # Resize frames to the common size before calculating metrics
        pred_frame = cv2.resize(pred_frame, common_size)
        gt_frame = cv2.resize(gt_frame, common_size)

        pred_gray = cv2.cvtColor(pred_frame, cv2.COLOR_BGR2GRAY)
        gt_gray = cv2.cvtColor(gt_frame, cv2.COLOR_BGR2GRAY)

        ssim_value = ssim(pred_gray, gt_gray)
        ssim_values.append(ssim_value)

        mse = np.mean((pred_gray - gt_gray) ** 2)
        if mse == 0:
            psnr_value = float('inf')
        else:
            psnr_value = 20 * np.log10(255.0 / np.sqrt(mse))
        psnr_values.append(psnr_value)

    avg_ssim = np.mean(ssim_values)
    avg_psnr = np.mean(psnr_values)

    return avg_ssim, avg_psnr, ssim_values, psnr_values

def extract_frames(video_path):
    cap = cv2.VideoCapture(video_path)
    frames = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(frame)

    cap.release()
    return frames

# Upload predicted video
print("Please upload the predicted video:")
uploaded_predicted = files.upload()
predicted_video_path = list(uploaded_predicted.keys())[0]

# Upload ground truth video
print("Please upload the ground truth video:")
uploaded_ground_truth = files.upload()
ground_truth_video_path = list(uploaded_ground_truth.keys())[0]

# Calculate SSIM and PSNR per frame
avg_ssim, avg_psnr, ssim_values, psnr_values = calculate_ssim_psnr_per_frame(predicted_video_path, ground_truth_video_path)
print(f"Average SSIM: {avg_ssim}")
print(f"Average PSNR: {avg_psnr}")

# Plot SSIM
plt.figure(figsize=(12, 6))
plt.plot(ssim_values, label='SSIM', color='g')
plt.xlabel('Frame')
plt.ylabel('SSIM')
plt.title('SSIM per Frame')
plt.legend()
plt.show()

# Plot PSNR
plt.figure(figsize=(12, 6))
plt.plot(psnr_values, label='PSNR', color='r')
plt.xlabel('Frame')
plt.ylabel('PSNR (dB)')
plt.title('PSNR per Frame')
plt.legend()
plt.show()
